In [1]:
import pandas as pd

def price_gas_storage_contract(
    injection_dates,
    withdrawal_dates,
    price_curve,               # dict or pandas Series: date -> price
    injection_rate,             # MMBtu per day
    withdrawal_rate,            # MMBtu per day
    max_storage,                # MMBtu
    storage_cost_per_day,       # $
    injection_cost_per_mmbtu,   # $
    withdrawal_cost_per_mmbtu   # $
):
    stored_volume = 0.0
    total_value = 0.0
    storage_days = 0

    # --- Injection phase ---
    for date in injection_dates:
        price = price_curve[pd.to_datetime(date)]

        inject_volume = min(
            injection_rate,
            max_storage - stored_volume
        )

        stored_volume += inject_volume
        total_value -= inject_volume * price
        total_value -= inject_volume * injection_cost_per_mmbtu

    # --- Storage cost ---
    if injection_dates and withdrawal_dates:
        start = pd.to_datetime(injection_dates[0])
        end = pd.to_datetime(withdrawal_dates[-1])
        storage_days = (end - start).days

    total_value -= storage_days * storage_cost_per_day

    # --- Withdrawal phase ---
    for date in withdrawal_dates:
        price = price_curve[pd.to_datetime(date)]

        withdraw_volume = min(
            withdrawal_rate,
            stored_volume
        )

        stored_volume -= withdraw_volume
        total_value += withdraw_volume * price
        total_value -= withdraw_volume * withdrawal_cost_per_mmbtu

    return total_value


In [2]:
# Sample price curve
price_curve = {
    pd.Timestamp("2024-06-30"): 2.0,
    pd.Timestamp("2024-07-31"): 2.1,
    pd.Timestamp("2024-12-31"): 3.0,
    pd.Timestamp("2025-01-31"): 3.2,
}

injection_dates = ["2024-06-30", "2024-07-31"]
withdrawal_dates = ["2024-12-31", "2025-01-31"]

contract_value = price_gas_storage_contract(
    injection_dates=injection_dates,
    withdrawal_dates=withdrawal_dates,
    price_curve=price_curve,
    injection_rate=500_000,
    withdrawal_rate=500_000,
    max_storage=1_000_000,
    storage_cost_per_day=3_000,
    injection_cost_per_mmbtu=0.01,
    withdrawal_cost_per_mmbtu=0.01
)

print(f"Contract Value: ${contract_value:,.0f}")


Contract Value: $385,000
